In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
map_ds = xr.open_dataset("/N/slate/jmelms/projects/ML_atmos_model_conservation_tests/Regrid_E3SMv2.1_to_360x180/Grids_and_Maps/map_ne30pg2_to_cmip6_180x360_aave.20200201.nc")
map_ds

<xarray.Dataset> Size: 11MB
Dimensions:        (src_grid_rank: 1, dst_grid_rank: 2, n_a: 21600, n_b: 64800,
                    nv_a: 4, nv_b: 4, n_s: 156107)
Dimensions without coordinates: src_grid_rank, dst_grid_rank, n_a, n_b, nv_a,
                                nv_b, n_s
Data variables: (12/19)
    src_grid_dims  (src_grid_rank) int32 4B ...
    dst_grid_dims  (dst_grid_rank) int32 8B ...
    yc_a           (n_a) float64 173kB ...
    yc_b           (n_b) float64 518kB ...
    xc_a           (n_a) float64 173kB ...
    xc_b           (n_b) float64 518kB ...
    ...             ...
    area_b         (n_b) float64 518kB ...
    frac_a         (n_a) float64 173kB ...
    frac_b         (n_b) float64 518kB ...
    col            (n_s) int32 624kB ...
    row            (n_s) int32 624kB ...
    S              (n_s) float64 1MB ...
Attributes: (12/15)
    title:               ESMF Offline Regridding Weight Generator
    normalization:       destarea
    map_method:          Conservative remapping
    ESMF_regrid_method:  First-order Conservative
    conventions:         NCAR-CSM
    domain_a:            /Users/zender/data/grids/ne30pg2.nc
    ...                  ...
    CVS_revision:        7.1.0r
    history:             Thu Jan 23 10:45:19 PST 2020: /Users/zender/bin/ncre...
    remap_script:        ncremap
    remap_hostname:      dhcp-v052-254.mobile.uci.edu
    remap_version:       4.9.2-alpha05
    NCO:                 netCDF Operators version 4.9.2-alpha05 (Homepage = h...

In [3]:
map_ds["frac_b"]

<xarray.DataArray 'frac_b' (n_b: 64800)> Size: 518kB
[64800 values with dtype=float64]
Dimensions without coordinates: n_b
Attributes:
    units:    unitless

In [4]:
high_res_grid = xr.open_dataset("/N/slate/jmelms/projects/ML_atmos_model_conservation_tests/Regrid_E3SMv2.1_to_360x180/Grids_and_Maps/r0125_1440x2880.20210401.nc")
high_res_grid

<xarray.Dataset> Size: 382MB
Dimensions:          (grid_rank: 2, grid_size: 4147200, grid_corners: 4)
Dimensions without coordinates: grid_rank, grid_size, grid_corners
Data variables:
    grid_dims        (grid_rank) int32 8B ...
    grid_area        (grid_size) float64 33MB ...
    grid_imask       (grid_size) int32 17MB ...
    grid_center_lat  (grid_size) float64 33MB ...
    grid_center_lon  (grid_size) float64 33MB ...
    grid_corner_lat  (grid_size, grid_corners) float64 133MB ...
    grid_corner_lon  (grid_size, grid_corners) float64 133MB ...
Attributes:
    title:                MOSART 1440x2880
    Conventions:          SCRIP
    created_by:           zender
    grid_generator:       NCO
    history:              Tue Mar  9 10:13:56 2021: ncks -O --dmm_in_mk --fl_...
    NCO:                  netCDF Operators version 4.9.8-beta01 (Homepage = h...
    latitude_grid_type:   Uniform/Equi-Angular latitude grid. Uniform/Equi-an...
    longitude_grid_type:  Date line at west edge of first longitude cell

In [ ]:
high_res_grid["grid_size"].values
ul_lat = high_res_grid["grid_corner_lat"].values[:, 3]
ur_lat = high_res_grid["grid_corner_lat"].values[:, 2]
ll_lat = high_res_grid["grid_corner_lat"].values[:, 0]
lr_lat = high_res_grid["grid_corner_lat"].values[:, 1]
ul_lon = high_res_grid["grid_corner_lon"].values[:, 3]
ur_lon = high_res_grid["grid_corner_lon"].values[:, 2]
ll_lon = high_res_grid["grid_corner_lon"].values[:, 0]
lr_lon = high_res_grid["grid_corner_lon"].values[:, 1]
m_lat_est = 0.25 * (ul_lat + ur_lat + ll_lat + lr_lat)
m_lon_est = 0.25 * (ul_lon + ur_lon + ll_lon + lr_lon)
m_lat = high_res_grid["grid_center_lat"].values
m_lon = high_res_grid["grid_center_lon"].values

In [17]:
print(f"If following values are (0, 0), your lat/lon centers derived from corners are equiv to given centers: {float((m_lat - m_lat_est).max()), float((m_lon - m_lon_est).max())}")

If following values are (0, 0), your lat/lon centers derived from corners are equiv to given centers: (0.0, 0.0)


In [30]:
print("Making lat/lon centers from given resolution and extents:")
res = 0.125 # degrees
lat_start = -90
lon_start = -180
lat_stop = 90
lon_stop = 180
nlat = int((lat_stop - lat_start) / res)
nlon = int((lon_stop - lon_start) / res)
m_lat_est = np.arange(lat_start+res/2, lat_stop+res/2, res).repeat(nlon) # lats look like -90, -90, -90, ..., 90, 90, 90
m_lon_est = np.tile(np.arange(lon_start+res/2, lon_stop+res/2, res), nlat) # lons look like -180, -179.875, ..., 179.875, 180, -180, -179.875, ...
print("Given lat/lon centers:")
print(m_lat)
print(m_lon)
print(f"Derived lat/lon centers:")
print(m_lat_est)
print(m_lon_est)

Making lat/lon centers from given resolution and extents:
Given lat/lon centers:
[-89.9375 -89.9375 -89.9375 ...  89.9375  89.9375  89.9375]
[-179.9375 -179.8125 -179.6875 ...  179.6875  179.8125  179.9375]
Derived lat/lon centers:
[-89.9375 -89.9375 -89.9375 ...  89.9375  89.9375  89.9375]
[-179.9375 -179.8125 -179.6875 ...  179.6875  179.8125  179.9375]


In [26]:
print(f"Now we'll try to make the corners from the centers and resolution, and see if they match the given corners.")
ul_lat_est = m_lat_est + res / 2
ur_lat_est = m_lat_est + res / 2
ll_lat_est = m_lat_est - res / 2
lr_lat_est = m_lat_est - res / 2
ul_lon_est = m_lon_est - res / 2
ur_lon_est = m_lon_est + res / 2
ll_lon_est = m_lon_est - res / 2
lr_lon_est = m_lon_est + res / 2
high_res_grid["grid_corner_lat_est"] = (("grid_size", "grid_corners"), np.array([ll_lat_est, lr_lat_est, ur_lat_est, ul_lat_est]).T)
high_res_grid["grid_corner_lon_est"] = (("grid_size", "grid_corners"), np.array([ll_lon_est, lr_lon_est, ur_lon_est, ul_lon_est]).T)
print(f"If following values are (0, 0), your corner lats/lons derived from centers and resolution are equiv to given corners: {float((high_res_grid['grid_corner_lat'].values - high_res_grid['grid_corner_lat_est'].values).max()), float((high_res_grid['grid_corner_lon'].values - high_res_grid['grid_corner_lon_est'].values).max())}")

Now we'll try to make the corners from the centers and resolution, and see if they match the given corners.
If following values are (0, 0), your corner lats/lons derived from centers and resolution are equiv to given corners: (0.0, 0.0)


In [27]:
print("Given latitude corners:")
print(high_res_grid["grid_corner_lat"].values[0:3])
print("Estimated latitude corners:")
print(high_res_grid["grid_corner_lat_est"].values[0:3])
print("Given longitude corners:")
print(high_res_grid["grid_corner_lon"].values[0:3])
print("Estimated longitude corners:")
print(high_res_grid["grid_corner_lon_est"].values[0:3])

Given latitude corners:
[[-90.    -90.    -89.875 -89.875]
 [-90.    -90.    -89.875 -89.875]
 [-90.    -90.    -89.875 -89.875]]
Estimated latitude corners:
[[-90.    -90.    -89.875 -89.875]
 [-90.    -90.    -89.875 -89.875]
 [-90.    -90.    -89.875 -89.875]]
Given longitude corners:
[[-180.    -179.875 -179.875 -180.   ]
 [-179.875 -179.75  -179.75  -179.875]
 [-179.75  -179.625 -179.625 -179.75 ]]
Estimated longitude corners:
[[-180.    -179.875 -179.875 -180.   ]
 [-179.875 -179.75  -179.75  -179.875]
 [-179.75  -179.625 -179.625 -179.75 ]]


In [33]:
high_res_grid["grid_area"].values.min(), high_res_grid["grid_area"].values.max()

(np.float64(5.191967603009097e-09), np.float64(4.759643408461334e-06))